In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import googlemaps

# Creating DataFrame polos

In [2]:
URL = "http://vestibular.univesp.br/polos"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [3]:
test = soup.findAll('li', attrs = {'class':'list-iconed-item'})
quotes=[]
for row in test: 
    quote = {} 
    quote['info'] = row.text
    quotes.append(quote)

polos = pd.DataFrame(quotes)
polos.sample(5)

,info
109,\n Guariba | 10 vagas por eixo (Lic...
92,\n Fartura | 10 vagas por eixo (Lic...
26,\n Bady Bassitt | 10 vagas por eixo...
342,\n Tremembé | 15 vagas por eixo (Li...
161,\n Laranjal Paulista | 10 vagas por...


In [4]:

polos["info"] = polos["info"].str.replace("\n ","",regex=True)

polos["polo"]     = polos["info"].str.split(" \| ", n = 1, expand = True)[0].str.strip()
polos["vagas_temp"] = polos["info"].str.split(" \| ", n = 1, expand = True)[1]

polos["vagas"]=polos["vagas_temp"].str.split("vagas por eixo \(Licenciaturas \/ Computação\)",n=1,expand=True)[0]
polos["endereco"]=polos["vagas_temp"].str.split("vagas por eixo \(Licenciaturas \/ Computação\)",n=1,expand=True)[1].str.strip()

polos.head()

,info,polo,vagas_temp,vagas,endereco
0,Adamantina | 10 vagas por eixo (Lic...,Adamantina,10 vagas por eixo (Licenciaturas / Computação)...,10,"Alameda dos Expedicionários, 864 - Centro CEP:..."
1,Aguaí | 10 vagas por eixo (Licencia...,Aguaí,10 vagas por eixo (Licenciaturas / Computação)...,10,"Av. Olinda Silveira Cruz Braga, 200 - Parque I..."
2,Agudos | 10 vagas por eixo (Licenci...,Agudos,10 vagas por eixo (Licenciaturas / Computação)...,10,"R. João Pessoa, 2 - Professor Simões CEP: 1712..."
3,Altinópolis | 10 vagas por eixo (Li...,Altinópolis,10 vagas por eixo (Licenciaturas / Computação)...,10,"R. Alagoas, 248 - Vila Maria CEP: 14350-000"
4,Alumínio | 10 vagas por eixo (Licen...,Alumínio,10 vagas por eixo (Licenciaturas / Computação)...,10,"Av. Senador José Ermírio de Moraes, 798 - Vila..."


In [5]:
polos = polos[['polo', 'vagas', 'endereco']]
polos.sample(5)

,polo,vagas,endereco
22,Arujá,30,"R. Mamede Barbosa, 105 - Centro CEP: 07400 - 820"
68,Cesário Lange,10,"R. Camargo Barros, 1210 - Centro CEP: 18285-000"
66,Cerqueira César,10,"Rod. Donato Francisco Sassi, Km 5 - Macucão CE..."
220,Porangaba,10,"R. Lazinho Nunes da Silva, 505 - Vista Alegre ..."
335,Taquarivai,10,"R. Maria Joana Ferraz, 280 - Centro CEP: 18425..."


In [7]:
from key_google import *  
for address in polos['endereco']:   
    gmaps = googlemaps.Client(key=key)
    try:
        geocode_result = gmaps.geocode(address)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        distance = gmaps.distance_matrix([str(lat_home) + " " + \
                                  str(lng_home)], \
                                 [str(lat) + " " + \
                                  str(lng)])['rows'][0]['elements'][0]['distance']['value']
        
        polos.loc[polos['endereco']==address,'lat'] = lat
        polos.loc[polos['endereco']==address,'lng'] = lng
        polos.loc[polos['endereco']==address,'distance'] = distance
        print('working - ' + address)

    except:
        print('error - ' + address)
        pass

working - Alameda dos Expedicionários, 864 - Centro CEP: 17800-000
working - Av. Olinda Silveira Cruz Braga, 200 - Parque Interlagos CEP: 13860-000
working - R. João Pessoa, 2 - Professor Simões CEP: 17120-000
working - R. Alagoas, 248 - Vila Maria CEP: 14350-000
working - Av. Senador José Ermírio de Moraes, 798 - Vila Industrial CEP: 18125-000
working - R. Vereador Antonio Teruel, 670 - Centro CEP: 17430-000
working - R. das Hortências, 1555 - Cidade Jardim CEP: 13467-190
working - Av. Nove de Julho, 192 - Centro CEP: 14820-000
working - R. Gustavo de Souza, 108 - Ribeirão Amparo CEP: 13904-903
working - R. Humberto de Campos, 229 - Centro CEP: 16901-012
working - R. Prefeito Spedito Scalise , S/N - Morada do Sol CEP: 18620-000
working - Praça Pe. Victor Coelho de Almeida, 100 - Jardim São Paulo CEP: 12570-000
working - R. Amarildo Gorgonha da Silva, 112 - Centro CEP: 18320-000
working - R. Amelia da Costa e Silva, 66 - Jardim Bela Vista CEP: 18147-000
working - R. Conselheiro Crispin

working - Av. Professor Malaquias de Oliveira Freitas, 95 - Barra Velha CEP: 11630-000
working - R. Dom Pedro I, 65 - Cidade Nova CEP: 13334-100
working - R. Domingos Fernandes, 240 - Centro CEP: 18950-000
working - R. Luiz Rossi, 107 - Jardim Irene CEP: 18560-000
working - R. Araripe Custódio da Fonseca, 775 - Altos de Ipeúna CEP: 13537-000
working - R. Pedro Silva, S/N - Centro CEP: 18330-000
working - R. Josephina Silva Melo, 365 - Centro CEP: 18440-000
working - R. Abílio Raposo Ferraz, 405 - Vila Jardim Brasil CEP: 18730-000
working - R. Cincinato Braga, 240 - Centro CEP: 15840-000
working - R. Antonio de Antonio, 70 - Centro CEP: 17260-000
working - Av. Condessa de Vimieiros, 1131 - Centro CEP: 11740-000
error - R. Sete de Setembro, 11 - Centro CEP: 18360-000
working - R. São João, 47 - Centro CEP: 06850-060
working - Av. Dr Cyro de Albuquerque, 4750 - Taboãozinho CEP: 18213-615
working - R. Lucas de Camargo, 419 - Camargo CEP: 18400-340
working - R. Professor Irineu Chaluppe, 65

working - R. Nove de Julho, 1100 - Centro CEP: 14390-000
working - R. Sete de Setembro, 34 - Setembro CEP: 11013-350
working - Av. Dom Jaime de Barros Câmara, 201 - Planalto CEP: 09895-400
error - Av. Goiás, 3400 - Santa Maria CEP: 09560-400
working - R. São Sebastião, 2828 - Aparque Santa Mônica CEP: 13561-207
working - R. Riachuelo, 444 - Centro CEP: 13870-234
working - Av. Fortunato Lobão, S/N - Centro CEP: 12830-000
working - R. General Glicério, 3947 - Vila Redentora CEP: 15015-400
working - Av. Dr. Altino Bondensan, 500 - Eugêncio de Melo CEP: 12247-016
working - Av. Olivo Gomes, 250 - Santana CEP: 12211-115
working - R. João Alfredo Guimarães, 386 - Centro CEP: 06890-000
working - R. José de Moraes Terra, 280 - Jardim Nova Vila Rica CEP: 18230-000
working - R. Bento Guelfi, 1802 - Jd Laranjeira CEP: 08381-001
working - R. Engº Armando de Arruda Pereira, 5241 - Jd Lourdes CEP: 04325-001
working - Av. Cantos do Amanhecer, s/nº - Jardim Eledy CEP: 5856020
working - R. Feitiço da Vi

In [ ]:
polos.sample(5)

In [ ]:
polos.to_csv('polos.csv')

In [8]:
import gmplot
